In [11]:
### Import of libs

from pyspark.sql.functions import current_date, current_timestamp, expr

StatementMeta(, e6f3d1b0-2cee-41bb-98d2-3fd5ad1d1e8e, 13, Finished, Available, Finished)

In [12]:
%%sql

-- Tabela 1, organizando por WBS

WITH all_wbs AS 
(
SELECT
    *
FROM delta.`Tables/Silver/Zone1/silver_Zone1`
UNION
SELECT
    *
FROM delta.`Tables/Silver/Zone2/silver_Zone2`
UNION
SELECT
    *
FROM delta.`Tables/Silver/Zone3/silver_Zone3`
)
SELECT 
    a.WBS,
    a.sap_internal_number,
    io.Project_BA,
    a.SAP_source,
    a.Currency,
    a.Year,
    a.BGT_LC,
    a.ACT_LC,
    a.COM_LC,
    a.Source_name AS Source_Name_SAP,
    a.load_time_GMT AS load_time_GMT_SAP,
    io.Source_Name AS Source_Name_Project,
    io.load_time_GMT AS load_time_GMT_Project
FROM all_wbs a
LEFT JOIN delta.`Tables/Silver/IO_Project/silver_IO_Project` io 
ON io.sap_internal_number = a.sap_internal_number
ORDER BY Project_BA ASC

StatementMeta(, e6f3d1b0-2cee-41bb-98d2-3fd5ad1d1e8e, 14, Finished, Available, Finished)

<Spark SQL result set with 12 rows and 13 fields>

In [13]:
%%sql

-- Tabela 2, organizando por Projeto

WITH wbs_cons AS
(
WITH all_wbs AS 
(
SELECT
    *
FROM delta.`Tables/Silver/Zone1/silver_Zone1`
UNION
SELECT
    *
FROM delta.`Tables/Silver/Zone2/silver_Zone2`
UNION
SELECT
    *
FROM delta.`Tables/Silver/Zone3/silver_Zone3`
)
SELECT 
    a.WBS,
    a.sap_internal_number,
    io.Project_BA,
    a.SAP_source,
    a.Currency,
    a.Year,
    a.BGT_LC,
    a.ACT_LC,
    a.COM_LC,
    a.Source_name AS Source_Name_SAP,
    a.load_time_GMT AS load_time_GMT_SAP,
    io.Source_Name AS Source_Name_Project,
    io.load_time_GMT AS load_time_GMT_Project
FROM all_wbs a
LEFT JOIN delta.`Tables/Silver/IO_Project/silver_IO_Project` io 
ON io.sap_internal_number = a.sap_internal_number
ORDER BY Project_BA ASC
)
SELECT
    Project_BA,
    Currency,
    Year,
    ROUND(SUM(BGT_LC),2) AS BGT_LC,
    ROUND(SUM(ACT_LC),2) AS ACT_LC,
    ROUND(SUM(COM_LC),2) AS COM_LC,
    ROUND(SUM(ACT_LC)+SUM(COM_LC),2) AS LE_LC,
    ROUND(SUM(BGT_LC) - SUM(ACT_LC) - SUM(COM_LC),2) AS Organic_LC
FROM wbs_cons
GROUP BY Project_BA, Currency, Year
ORDER BY Project_BA ASC


StatementMeta(, e6f3d1b0-2cee-41bb-98d2-3fd5ad1d1e8e, 15, Finished, Available, Finished)

<Spark SQL result set with 9 rows and 8 fields>

In [14]:
## Gold Table creation - WBS level

### Table Creation

df = spark.sql(
f"""
WITH all_wbs AS 
(
SELECT
    *
FROM delta.`Tables/Silver/Zone1/silver_Zone1`
UNION
SELECT
    *
FROM delta.`Tables/Silver/Zone2/silver_Zone2`
UNION
SELECT
    *
FROM delta.`Tables/Silver/Zone3/silver_Zone3`
)
SELECT 
    a.WBS,
    a.sap_internal_number,
    io.Project_BA,
    a.SAP_source,
    a.Currency,
    a.Year,
    a.BGT_LC,
    a.ACT_LC,
    a.COM_LC,
    a.Source_name AS Source_Name_SAP,
    a.load_time_GMT AS load_time_GMT_SAP,
    io.Source_Name AS Source_Name_Project,
    io.load_time_GMT AS load_time_GMT_Project
FROM all_wbs a
LEFT JOIN delta.`Tables/Silver/IO_Project/silver_IO_Project` io 
ON io.sap_internal_number = a.sap_internal_number
ORDER BY Project_BA ASC
"""
)

df = df.withColumn("load_time_GMT", current_timestamp())

### Note that it is GMT. Please see Layer bronze for further details.

# Caminho do destino
caminho = "abfss://SAP_Ingest_v2@onelake.dfs.fabric.microsoft.com/Gold.Lakehouse/Files/Gold/WBS_Level/Gold_WBS_Level"


# Salvando o DataFrame no formato Delta
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(caminho)

print("Dados salvos com sucesso!")

# Executando o comando OPTIMIZE para otimizar a tabela Delta
spark.sql(f"OPTIMIZE delta.`{caminho}`")

# Confirmação de que a otimização foi concluída
print(f"Optimization for {caminho} completed!")

StatementMeta(, e6f3d1b0-2cee-41bb-98d2-3fd5ad1d1e8e, 16, Finished, Available, Finished)

Dados salvos com sucesso!
Optimization for abfss://SAP_Ingest_v2@onelake.dfs.fabric.microsoft.com/Gold.Lakehouse/Files/Gold/WBS_Level/Gold_WBS_Level completed!


In [15]:
## Gold Table creation - Project level

### Table Creation

df = spark.sql(
f"""
WITH wbs_cons AS
(
WITH all_wbs AS 
(
SELECT
    *
FROM delta.`Tables/Silver/Zone1/silver_Zone1`
UNION
SELECT
    *
FROM delta.`Tables/Silver/Zone2/silver_Zone2`
UNION
SELECT
    *
FROM delta.`Tables/Silver/Zone3/silver_Zone3`
)
SELECT 
    a.WBS,
    a.sap_internal_number,
    io.Project_BA,
    a.SAP_source,
    a.Currency,
    a.Year,
    a.BGT_LC,
    a.ACT_LC,
    a.COM_LC,
    a.Source_name AS Source_Name_SAP,
    a.load_time_GMT AS load_time_GMT_SAP,
    io.Source_Name AS Source_Name_Project,
    io.load_time_GMT AS load_time_GMT_Project
FROM all_wbs a
LEFT JOIN delta.`Tables/Silver/IO_Project/silver_IO_Project` io 
ON io.sap_internal_number = a.sap_internal_number
ORDER BY Project_BA ASC
)
SELECT
    Project_BA,
    Currency,
    Year,
    ROUND(SUM(BGT_LC),2) AS BGT_LC,
    ROUND(SUM(ACT_LC),2) AS ACT_LC,
    ROUND(SUM(COM_LC),2) AS COM_LC,
    ROUND(SUM(ACT_LC)+SUM(COM_LC),2) AS LE_LC,
    ROUND(SUM(BGT_LC) - SUM(ACT_LC) - SUM(COM_LC),2) AS Organic_LC
FROM wbs_cons
GROUP BY Project_BA, Currency, Year
ORDER BY Project_BA ASC
"""
)

df = df.withColumn("load_time_GMT", current_timestamp())

### Note that it is GMT. Please see Layer bronze for further details.

# Caminho do destino
caminho = "abfss://SAP_Ingest_v2@onelake.dfs.fabric.microsoft.com/Gold.Lakehouse/Files/Gold/Project_Level/Gold_Project_Level"


# Salvando o DataFrame no formato Delta
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(caminho)

print("Dados salvos com sucesso!")

# Executando o comando OPTIMIZE para otimizar a tabela Delta
spark.sql(f"OPTIMIZE delta.`{caminho}`")

# Confirmação de que a otimização foi concluída
print(f"Optimization for {caminho} completed!")

StatementMeta(, e6f3d1b0-2cee-41bb-98d2-3fd5ad1d1e8e, 17, Finished, Available, Finished)

Dados salvos com sucesso!
Optimization for abfss://SAP_Ingest_v2@onelake.dfs.fabric.microsoft.com/Gold.Lakehouse/Files/Gold/Project_Level/Gold_Project_Level completed!
